In [53]:
import os
ionq_token='LAYq6EwdvXqh4sIma2iKZNQ2l7KzbxX6'
from qiskit_ionq import IonQProvider
from threading import Thread
provider = IonQProvider(ionq_token)

# show supported backends
print(provider.backends()) 

# create backends for simulator and hardware
backend_simq = provider.get_backend("ionq_simulator")
backend_qpu = provider.get_backend("ionq_qpu")

[<IonQSimulatorBackend('ionq_simulator')>, <IonQQPUBackend('ionq_qpu')>]


In [ ]:
from qiskit_ionq import GPIGate, GPI2Gate, MSGate
backend_simq2 = provider.get_backend("ionq_simulator", gateset="native")
backend_qpu2 = provider.get_backend("ionq_qpu", gateset="native")

backend_simq = provider.get_backend("ionq_simulator")
backend_qpu = provider.get_backend("ionq_qpu")

In [ ]:
def gen_native_calib_circs(qn):
    qc = QuantumCircuit(qn)
    circ_list = []
    build_native_circ(qn, qc, circ_list)
    return circ_list

# build_circ: builds the circuits for gen_calib_circs
# IMPORTANT NOTE: THIS DOES NOT MEASURE ALL BY DEFAULT
def build_native_circ(i, qc, circ_list):
    if i > 0:
        build_native_circ(i-1, qc.copy(), circ_list)
        qc.append(GPIGate(0.5), [i - 1])
        build_native_circ(i-1, qc, circ_list)
    else:
        circ_list.append(qc)
        
for i in range(8):
        print(gen_native_calib_circs(3)[i].draw())
        

In [22]:
#We are now going to generate bytes using IonQ's native gate-set which does not have built in error correction
#We can compare generation rates and statistical test results accordingly.

from qiskit import QuantumCircuit

qc = QuantumCircuit(1)

qc.append(GPI2Gate(0.25), [0])
qc.append(GPIGate(0.5), [0])

In [41]:
qpu_job = backend_simq2.run(qc, shots = 2**(18))

In [42]:
#Confirming that we indeed generated an equal superposition.

print(qpu_job.result().get_counts())

{'0': 130808, '1': 131336}


In [48]:
#Now we adapt our circuit to generate a byte.

qc = QuantumCircuit(8)

for i in range(8):
    qc.append(GPI2Gate(0.25), [i])
    qc.append(GPIGate(0.5), [i])    

In [49]:
def gen_byte():
    qpu_job = backend_qpu2.run(qc, shots = 1)
    counts = qpu_job.get_counts()
    byte = list(counts.keys())[0]
    return byte

In [63]:
from array import *
import os

def gen_2e19_bytes(bin_array):
    bits = open("IONQ_2e20_bytes_native.bin", "ab")

    for i in range(2**19):
        byte = gen_byte()
        bin_array.append(int(byte,2))

    bin_array.tofile(bits)

In [66]:
import time

start = time.time()

bin_array1 = array('B')
bin_array2 = array('B')

t1 = Thread(target = gen_2e19_bytes, args = [bin_array1])
t2 = Thread(target = gen_2e19_bytes, args = [bin_array2])

t1.start()
t2.start()

t1.join()
t2.join()

time = time.time() - start
print(time)
print(bin_array1)
print(bin_array2)

KeyboardInterrupt: 

In [67]:
time = time.time() - start

print("Time elapsed: ", time)
print("Length of binary array 1: ", len(bin_array1))
print("Length of binary array 2: ", len(bin_array2))
print("Bytes generated: ", (len(bin_array1) + len(bin_array2)))

print("Generation rate (bps): ", (len(bin_array1) + len(bin_array2))/(time))

Time elapsed:  33001.251157045364
Length of binary array 1:  66
Length of binary array 2:  67
Bytes generated:  133
Generation rate (bps):  0.004030150231792231


In [68]:
bits = open("IONQ_DATASET_072022_THREADED_NONNATIVE.bin", "ab")

bin_array1.tofile(bits)
bin_array2.tofile(bits)

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 421, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 416, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/http/client.py", line 1336, in getresponse
    response.begin()
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/http/client.py", line 306, in begin
    version, status, reason = self._read_status()
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/http/client.py", line 275, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisco

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 672, in urlopen
    chunked=chunked,
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 376, in _make_request
    self._validate_conn(conn)
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 994, in _validate_conn
    conn.connect()
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/site-packages/urllib3/connection.py", line 360, in connect
    ssl_context=context,
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/site-packages/urllib3/util/ssl_.py", line 370, in ssl_wrap_socket
    return context.wrap_socket(sock, server_hostname=server_hostname)
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/ssl.py", line 423, in wrap_socket
    session=session
  File "/Users/mcbookair/opt/anaconda3/lib/python3.7/ssl.py", line 870,